In [1]:
import pandas as pd
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
import math
from IPython.display import display
from sklearn.externals import joblib
import os
import shap
import helper_functions as helper

# Global variables needed
OUTPATH_RESULTS = "results/xgb/"
CV_FOLDS = 10
N_JOBS = 4
N_ITER = 80
SEED = 12
COLS_KEEP = ["rank_test_score", "mean_test_score", "std_test_score",
             "param_learning_rate", "param_gamma", "param_max_depth", "param_colsample_bytree", "param_subsample",
             "param_min_child_weight", "param_n_estimators", "param_reg_alpha", "param_reg_lambda"]

"""
If you have pre-trained the model, skip to the code section that retrieves the best model.
"""
X_train = pd.read_csv("train_test_data/X_train.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
X_test = pd.read_csv("train_test_data/X_test.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
y_test = pd.read_csv("train_test_data/y_test.csv", index_col="uid").drop("Unnamed: 0", axis="columns")
y_train = pd.read_csv("train_test_data/y_train.csv", index_col="uid").drop("Unnamed: 0", axis="columns")

In [5]:
"""
Retrieve our desired training data and test data
"""
# We only consider the post scores
class_names = [col for col in y_test.columns if "post" in col]
# Use the wk 9 to 10 training data only
used_weeks = [str(wk) for wk in range(9, 11)]
filtered_columns = X_train.columns[X_train.columns.str.endswith(tuple(used_weeks))]
# Select all wk 9 and 10 features
train_data = {"class": {}, "regr": {}}
test_data = {"class": {}, "regr": {}}
# Prepare training and test data for classification model and regression model respectively
for name in class_names:
    # Classification labels
    if "class" in name:
        target_type = "class"
    else:
        target_type = "regr"
    train_data[target_type][name] = {"x": {}, "y": {}}
    test_data[target_type][name] = {"x": {}, "y": {}}
    # Only consider data which has the post score being not null
    not_null_indices = np.logical_not(y_train.loc[:, name].isnull())
    train_data[target_type][name]["x"] = X_train[filtered_columns][not_null_indices]
    train_data[target_type][name]["y"] = y_train[not_null_indices].loc[:, name]
    test_data[target_type][name]["x"] = X_test[filtered_columns]
    test_data[target_type][name]["y"] = y_test.loc[:, name]

In [9]:
# Classifier parameter grid
class_parameters = {"learning_rate": [0.001, 0.01, 0.1], 
                    "gamma" : [0, 0.01, 0.05, 0.1],
                    "max_depth": [1, 3, 5, 7],
                    "colsample_bytree": [0.3, 0.5, 0.7, 1.0],
                    "subsample": [0.3, 0.5, 0.7, 1.0],
                    "min_child_weight": [1, 2, 3],
                    "n_estimators": [100, 300, 500, 1000, 1500],
                    "reg_alpha": [0],
                    "reg_lambda": [1]
                    }

In [10]:
# Function to find the best hyperparameters for each model
def find_best_model(rscv, x, y):
    best_model = rscv.fit(x, y)
    return best_model


def report_classifier_performance(model, x, y):
    predict = model.predict(x)
    predict_proba = model.predict_proba(x)[:, 1]
    print("Accuracy :", metrics.accuracy_score(y, predict))
    print("logloss:", metrics.log_loss(y, predict_proba))
    print("AUC Score:", metrics.roc_auc_score(y, predict_proba))
    print()

In [6]:
# from https://scikit-learn.org/stable/auto_examples/model_selection/plot_randomized_search.html
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [7]:
# Baseline classifier
baseline_classifier = XGBClassifier(objective='binary:logistic', scale_pos_weight=1)
rscv_classifier = RandomizedSearchCV(baseline_classifier, class_parameters,
                                     scoring="neg_log_loss", verbose=False, cv=CV_FOLDS,
                                     n_jobs=N_JOBS, n_iter=N_ITER, iid=False, random_state=SEED)
# Find the optimal classifier and its performance on training and test set respectively
for name in train_data["class"]:
    print("Prediction of " + name)
    best_model = find_best_model(rscv_classifier, train_data["class"][name]["x"], train_data["class"][name]["y"])
    # Save the best model in pickle
    joblib.dump(best_model, 'xgb_model_' + name)
    report(best_model.cv_results_)
    print("Performance on training set:")
    report_classifier_performance(best_model, train_data["class"][name]["x"], train_data["class"][name]["y"])
    print("Performance on test set:")
    report_classifier_performance(best_model, test_data["class"][name]["x"], test_data["class"][name]["y"])
    print("-------------------------------------------------------------------------------------------------\n")

Prediction of panas_pos_raw_class_post
Model with rank: 1
Mean validation score: -0.683 (std: 0.119)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 7, 'learning_rate': 0.001, 'gamma': 0.1, 'colsample_bytree': 0.7}

Model with rank: 2
Mean validation score: -0.684 (std: 0.076)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 300, 'min_child_weight': 2, 'max_depth': 1, 'learning_rate': 0.001, 'gamma': 0, 'colsample_bytree': 0.7}

Model with rank: 3
Mean validation score: -0.684 (std: 0.101)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 2, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 0.01, 'colsample_bytree': 0.5}

Performance on training set:


Accuracy : 0.9310344827586207
logloss: 0.5566060769146887
AUC Score: 0.9761904761904763

Performance on test set:
Accuracy : 0.7777777777777778
logloss: 0.6363573173681895
AUC Score: 0.5555555555555556

-------------------------------------------------------------------------------------------------

Prediction of panas_neg_raw_class_post
Model with rank: 1
Mean validation score: -0.600 (std: 0.162)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.001, 'gamma': 0.05, 'colsample_bytree': 0.5}

Model with rank: 2
Mean validation score: -0.642 (std: 0.128)
Parameters: {'subsample': 0.7, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 0.05, 'colsample_bytree': 0.7}

Model with rank: 3
Mean validation score: -0.642 (std: 0.080)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 500, 'min_chi

0.6346431440777249
AUC Score: 0.3333333333333333

-------------------------------------------------------------------------------------------------

Prediction of panas_neg_imp_class_post
Model with rank: 1
Mean validation score: -0.600 (std: 0.162)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.001, 'gamma': 0.05, 'colsample_bytree': 0.5}

Model with rank: 2
Mean validation score: -0.642 (std: 0.128)
Parameters: {'subsample': 0.7, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 3, 'learning_rate': 0.001, 'gamma': 0.05, 'colsample_bytree': 0.7}

Model with rank: 3
Mean validation score: -0.642 (std: 0.080)
Parameters: {'subsample': 1.0, 'reg_lambda': 1, 'reg_alpha': 0, 'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 1, 'learning_rate': 0.001, 'gamma': 0, 'colsample_bytree': 1.0}

Performance on training set:
Accuracy : 0.9333333333333333
l

 0.6666666666666666
logloss: 0.7197290940417184
AUC Score: 0.6

-------------------------------------------------------------------------------------------------



In [28]:
def report_regression_performance(model, x, y):
    predict = model.predict(x)
    print("MSE :", metrics.mean_squared_error(y, predict))
    print("RMSE :", math.sqrt(metrics.mean_squared_error(y, predict)))
    df = pd.DataFrame({'Actual': np.array(y).reshape(-1),
                       'Predicted': np.array(predict).reshape(-1)})
    print()
    return df


# Classifier parameter grid
regr_parameters = {"learning_rate": [0.001, 0.01, 0.1], 
                    "gamma" : [0, 0.01, 0.05, 0.1],
                    "max_depth": [1, 3, 5, 7],
                    "colsample_bytree": [0.3, 0.5, 0.7, 1.0],
                    "subsample": [0.3, 0.5, 0.7, 1.0],
                    "min_child_weight": [1, 2, 3],
                    "n_estimators": [100, 300, 500, 1000, 1500],
                    'reg_alpha': [1e-5, 1e-2,  0.75],
                    'reg_lambda': [1e-5, 1e-2, 0.45],
                   }

In [59]:
# Baseline regressor
baseline_regressor = XGBRegressor(objective="reg:squarederror")
xgb_rscv_regressor = RandomizedSearchCV(baseline_regressor, param_distributions=regr_parameters, 
                                        scoring="neg_mean_squared_error", iid=False, random_state=12,
                                        cv=CV_FOLDS, verbose=False, n_jobs=N_JOBS, n_iter=N_ITER)

# Find the optimal classifier and its performance on training and test set respectively
for name in train_data["regr"]:
    print("Prediction of " + name)
    best_model = find_best_model(xgb_rscv_regressor, train_data["regr"][name]["x"], train_data["regr"][name]["y"])
    # Save the best model in pickle
    joblib.dump(best_model, 'xgb_model_' + name)
    report(best_model.cv_results_)
    print("Performance on training set:")
    report_regression_performance(best_model, train_data["regr"][name]["x"], train_data["regr"][name]["y"])
    print("Performance on test set:")
    report_regression_performance(best_model, test_data["regr"][name]["x"], test_data["regr"][name]["y"])
    print("-------------------------------------------------------------------------------------------------\n")

Prediction of panas_pos_raw_post


C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model with rank: 1
Mean validation score: -36.292 (std: 19.139)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.75, 'n_estimators': 1500, 'min_child_weight': 3, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.5}

Model with rank: 2
Mean validation score: -36.822 (std: 20.410)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.01, 'n_estimators': 1500, 'min_child_weight': 2, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.05, 'colsample_bytree': 0.5}

Model with rank: 3
Mean validation score: -36.950 (std: 19.361)
Parameters: {'subsample': 0.5, 'reg_lambda': 0.45, 'reg_alpha': 0.75, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.05, 'colsample_bytree': 0.7}

Performance on training set:
MSE : 0.2880797044097813
RMSE : 0.5367305696620803

Performance on test set:
MSE : 41.650394343976785
RMSE : 6.453711671896784

-----------------------------------------------------------------

C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model with rank: 1
Mean validation score: -39.667 (std: 14.479)
Parameters: {'subsample': 0.7, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.3}

Model with rank: 2
Mean validation score: -42.825 (std: 18.180)
Parameters: {'subsample': 1.0, 'reg_lambda': 1e-05, 'reg_alpha': 1e-05, 'n_estimators': 1500, 'min_child_weight': 2, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.3}

Model with rank: 3
Mean validation score: -44.452 (std: 17.808)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 2, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.5}

Performance on training set:
MSE : 0.0014567241151477598
RMSE : 0.03816705536385745

Performance on test set:
MSE : 82.3441906877504
RMSE : 9.074369988475805

-------------------------------------------------------------

...

Model with rank: 1
Mean validation score: -34.092 (std: 16.251)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.75, 'n_estimators': 1500, 'min_child_weight': 3, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.5}

Model with rank: 2
Mean validation score: -34.462 (std: 15.739)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.01, 'n_estimators': 1500, 'min_child_weight': 2, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.05, 'colsample_bytree': 0.5}

Model with rank: 3
Mean validation score: -36.885 (std: 15.751)
Parameters: {'subsample': 0.5, 'reg_lambda': 0.45, 'reg_alpha': 0.75, 'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.05, 'colsample_bytree': 0.7}

Performance on training set:
MSE : 0.27205581612021584
RMSE : 0.5215897009338047

Performance on test set:
MSE : 40.341642352398694
RMSE : 6.35150709299759

-----------------------------------------------------------------

C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


Model with rank: 1
Mean validation score: -39.667 (std: 14.479)
Parameters: {'subsample': 0.7, 'reg_lambda': 0.01, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 5, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.3}

Model with rank: 2
Mean validation score: -42.825 (std: 18.180)
Parameters: {'subsample': 1.0, 'reg_lambda': 1e-05, 'reg_alpha': 1e-05, 'n_estimators': 1500, 'min_child_weight': 2, 'max_depth': 3, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.3}

Model with rank: 3
Mean validation score: -44.452 (std: 17.808)
Parameters: {'subsample': 0.7, 'reg_lambda': 1e-05, 'reg_alpha': 0.01, 'n_estimators': 1000, 'min_child_weight': 2, 'max_depth': 7, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.5}

Performance on training set:
MSE : 0.0014567241151477598
RMSE : 0.03816705536385745

Performance on test set:
MSE : 82.3441906877504
RMSE : 9.074369988475805

-------------------------------------------------------------

Model with rank: 1
Mean validation score: -89.423 (std: 60.355)
Parameters: {'subsample': 0.3, 'reg_lambda': 0.01, 'reg_alpha': 1e-05, 'n_estimators': 300, 'min_child_weight': 1, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.05, 'colsample_bytree': 0.3}

Model with rank: 2
Mean validation score: -99.495 (std: 81.597)
Parameters: {'subsample': 0.3, 'reg_lambda': 0.45, 'reg_alpha': 1e-05, 'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.1, 'colsample_bytree': 0.5}

Model with rank: 3
Mean validation score: -100.238 (std: 62.452)
Parameters: {'subsample': 0.3, 'reg_lambda': 0.45, 'reg_alpha': 0.01, 'n_estimators': 300, 'min_child_weight': 2, 'max_depth': 1, 'learning_rate': 0.01, 'gamma': 0.01, 'colsample_bytree': 0.7}

Performance on training set:
MSE : 36.46145106075879
RMSE : 6.038331811084812

Performance on test set:
MSE : 57.610016007438794
RMSE : 7.5901262181493925

-------------------------------------------------------------------

C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\xgboost\core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [2]:
# Retrieve best models
flourishing_scale_imp_class_post = joblib.load('xgb_model_flourishing_scale_imp_class_post')
flourishing_scale_raw_class_post = joblib.load('xgb_model_flourishing_scale_raw_class_post')
flourishing_scale_imp_post = joblib.load('xgb_model_flourishing_scale_imp_post')
flourishing_scale_raw_post = joblib.load('xgb_model_flourishing_scale_raw_post')
panas_neg_imp_post = joblib.load('xgb_model_panas_neg_imp_post')
panas_neg_raw_post = joblib.load('xgb_model_panas_neg_raw_post')
panas_pos_imp_post = joblib.load('xgb_model_panas_pos_imp_post')
panas_pos_raw_post = joblib.load('xgb_model_panas_pos_raw_post')
panas_neg_imp_class_post = joblib.load('xgb_model_panas_neg_imp_class_post')
panas_neg_raw_class_post = joblib.load('xgb_model_panas_neg_raw_class_post')
panas_pos_imp_class_post = joblib.load('xgb_model_panas_pos_imp_class_post')
panas_pos_raw_class_post = joblib.load('xgb_model_panas_pos_raw_class_post')

In [8]:
# Used to retrieve the grid search results
def combine_imp_raw_results(raw_cv_results, imp_cv_results, fpath_out):
    """
        takes the cv results from the randomised grid search object from the raw and imputed target models
        combines them, sorts by score and outputs
        """
    df_raw = pd.DataFrame(raw_cv_results)[COLS_KEEP]
    df_raw.insert(3, "param__target_imputation", "No Imputation")

    df_imp = pd.DataFrame(imp_cv_results)[COLS_KEEP]
    df_imp.insert(3, "param__target_imputation", "KNN Imputation")

    df = pd.concat([df_raw, df_imp]).sort_values("mean_test_score", ascending=False)
    df.rank_test_score = list(range(1, df.shape[0] + 1))
    df.columns = ["Rank", "CV score (mean)", "CV score (standard deviation)", 
                  "Target imputation", "Learning rate", "Gamma", 
                  "Max depth", "Max features in each tree (split)", "Subsample", 
                  "Min child weight", "Number of estimators (boost rounds)", "Alpha", "Lambda"]
    df["CV score (mean)"] = -df["CV score (mean)"]
    df.to_csv(fpath_out)
    return df

In [63]:
if not os.path.isdir(OUTPATH_RESULTS):
    os.mkdir(OUTPATH_RESULTS)
combine_imp_raw_results(flourishing_scale_raw_class_post.cv_results_,
                        flourishing_scale_imp_class_post.cv_results_,
                        OUTPATH_RESULTS + "flourishing_scale_class_post" + ".csv")
combine_imp_raw_results(flourishing_scale_raw_post.cv_results_,
                        flourishing_scale_imp_post.cv_results_,
                        OUTPATH_RESULTS + "flourishing_scale_post" + ".csv")

combine_imp_raw_results(panas_pos_raw_class_post.cv_results_,
                        panas_pos_imp_class_post.cv_results_,
                        OUTPATH_RESULTS + "panas_pos_class_post" + ".csv")
combine_imp_raw_results(panas_pos_raw_post.cv_results_,
                        panas_pos_imp_post.cv_results_,
                        OUTPATH_RESULTS + "panas_pos_post" + ".csv")
combine_imp_raw_results(panas_neg_raw_class_post.cv_results_,
                        panas_neg_imp_class_post.cv_results_,
                        OUTPATH_RESULTS + "panas_neg_class_post" + ".csv")
combine_imp_raw_results(panas_neg_raw_post.cv_results_,
                        panas_neg_imp_post.cv_results_,
                        OUTPATH_RESULTS + "panas_neg_post" + ".csv")

C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
C:\Users\brs97\AppData\Local\Programs\Python\Python37\lib\site-packages\sklearn\utils\deprecation.py:125: FutureWarning: You are accessing a training score ('split2_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.

,Rank,CV score (mean),CV score (standard deviation),Target imputation,Learning rate,Gamma,Max depth,Max features in each tree (split),Subsample,Min child weight,Number of estimators (boost rounds),Alpha,Lambda
13,1,39.667198,14.479418,KNN Imputation,0.01,0.01,5,0.3,0.7,1,1000,0.01,0.01
13,2,39.667198,14.479418,No Imputation,0.01,0.01,5,0.3,0.7,1,1000,0.01,0.01
22,3,42.825225,18.179539,KNN Imputation,0.01,0.01,3,0.3,1,2,1500,1e-05,1e-05
22,4,42.825225,18.179539,No Imputation,0.01,0.01,3,0.3,1,2,1500,1e-05,1e-05
3,5,44.451774,17.807880,No Imputation,0.01,0.01,7,0.5,0.7,2,1000,0.01,1e-05
3,6,44.451774,17.807880,KNN Imputation,0.01,0.01,7,0.5,0.7,2,1000,0.01,1e-05
41,7,44.548143,20.714477,KNN Imputation,0.01,0,1,0.3,1,2,300,1e-05,0.01
41,8,44.548143,20.714477,No Imputation,0.01,0,1,0.3,1,2,300,1e-05,0.01
23,9,47.382443,26.829384,No Imputation,0.01,0,7,0.3,0.7,2,300,0.75,0.45
23,10,47.382443,26.829384,KNN Imputation,0.01,0,7,0.3,0.7,2,300,0.75,0.45


In [3]:
# functions to output results
def output_results_df(cv_results, cols_keep, target):    
    pd.DataFrame(cv_results).sort_values("rank_test_score", ascending=True)[cols_keep].to_csv(OUTPATH_RESULTS + target + ".csv")


def output_results_diagnostics(estimator, target, X_train, y_train, X_test, y_test, quantiles=5, classifier=False, metric_name="Score"):
    y_train_pred = estimator.predict(X_train)
    y_test_pred = estimator.predict(X_test)
    if classifier:
        y_train_pred_proba = estimator.predict_proba(X_train)[:,1]
        y_test_pred_proba = estimator.predict_proba(X_test)[:,1]  
        score_train = metrics.log_loss(y_train, y_train_pred_proba)
        score_test = metrics.log_loss(y_test, y_test_pred_proba)
        metric_name = "Log Loss"
        helper.lorenz_curve(y_train, y_train_pred_proba)
        plt.savefig(OUTPATH_RESULTS + target + "_lorenz_train.png")
        helper.lorenz_curve(y_test, y_test_pred_proba)   
        plt.savefig(OUTPATH_RESULTS + target + "_lorenz_test.png")
    else:
        score_train = metrics.mean_squared_error(y_train, y_train_pred)
        score_test = metrics.mean_squared_error(y_test, y_test_pred)      
        metric_name = "MSE"
        helper.lorenz_curve(y_train, y_train_pred)
        plt.savefig(OUTPATH_RESULTS + target + "_lorenz_train.png")
        helper.lorenz_curve(y_test, y_test_pred)   
        plt.savefig(OUTPATH_RESULTS + target + "_lorenz_test.png")

    helper.quantile_plot(y_train, y_train_pred, quantiles=quantiles, title=metric_name + " train: {:.4f}".format(score_train))
    plt.savefig(OUTPATH_RESULTS + target + "_pvo_train.png")

    helper.quantile_plot(y_test, y_test_pred, quantiles=quantiles, title=metric_name + " test: {:.4f}".format(score_test))
    plt.savefig(OUTPATH_RESULTS + target + "_pvo_test.png")
        

def output_results_shap(estimator, target, X, corr_thresh=0.3):
    shap_values = shap.TreeExplainer(estimator).shap_values(X)
    shap.summary_plot(shap_values, X, plot_type="bar", show=False, max_display=10, color="orange")
    plt.savefig(OUTPATH_RESULTS + target + "_shap_bar.png", bbox_inches='tight')
    shap.summary_plot(shap_values, X, show=False, max_display=10)
    plt.savefig(OUTPATH_RESULTS + target + "_shap.png", bbox_inches='tight')


def output_results(estimator, cv_results, target, X_train, y_train, X_test, y_test, cols_keep, quantiles, classifier=False,
                  metric_name="Score"):
    plt.clf()
    output_results_df(cv_results, cols_keep, target)
    plt.clf()
    output_results_diagnostics(estimator, target, X_train, y_train, X_test, y_test, classifier=classifier, quantiles=quantiles, metric_name=metric_name)
    plt.clf()
    output_results_shap(estimator.best_estimator_, target, X_train)
    plt.clf()


In [ ]:
output_results(flourishing_scale_raw_class_post, flourishing_scale_raw_class_post.cv_results_, 
               "flourishing_scale_raw_class_post", train_data["class"]["flourishing_scale_raw_class_post"]["x"],
               train_data["class"]["flourishing_scale_raw_class_post"]["y"],
               test_data["class"]["flourishing_scale_raw_class_post"]["x"],
               test_data["class"]["flourishing_scale_raw_class_post"]["y"], COLS_KEEP, 5, classifier=True)

output_results(flourishing_scale_raw_post, flourishing_scale_raw_post.cv_results_, 
               "flourishing_scale_raw_post", train_data["regr"]["flourishing_scale_raw_post"]["x"],
               train_data["regr"]["flourishing_scale_raw_post"]["y"],
               test_data["regr"]["flourishing_scale_raw_post"]["x"],
               test_data["regr"]["flourishing_scale_raw_post"]["y"], COLS_KEEP, 5, classifier=False)

output_results(panas_pos_raw_class_post, panas_pos_raw_class_post.cv_results_, 
               "panas_pos_raw_class_post", train_data["class"]["panas_pos_raw_class_post"]["x"],
               train_data["class"]["panas_pos_raw_class_post"]["y"],
               test_data["class"]["panas_pos_raw_class_post"]["x"],
               test_data["class"]["panas_pos_raw_class_post"]["y"], COLS_KEEP, 5, classifier=True)

output_results(panas_pos_raw_post, panas_pos_raw_post.cv_results_, 
               "panas_pos_raw_post", train_data["regr"]["panas_pos_raw_post"]["x"],
               train_data["regr"]["panas_pos_raw_post"]["y"],
               test_data["regr"]["panas_pos_raw_post"]["x"],
               test_data["regr"]["panas_pos_raw_post"]["y"], COLS_KEEP, 5, classifier=False)

output_results(panas_neg_raw_class_post, panas_neg_raw_class_post.cv_results_, 
               "panas_neg_raw_class_post", train_data["class"]["panas_neg_raw_class_post"]["x"],
               train_data["class"]["panas_neg_raw_class_post"]["y"],
               test_data["class"]["panas_neg_raw_class_post"]["x"],
               test_data["class"]["panas_neg_raw_class_post"]["y"], COLS_KEEP, 5, classifier=True)

output_results(panas_neg_raw_post, panas_neg_raw_post.cv_results_, 
               "panas_neg_raw_post", train_data["regr"]["panas_neg_raw_post"]["x"],
               train_data["regr"]["panas_neg_raw_post"]["y"],
               test_data["regr"]["panas_neg_raw_post"]["x"],
               test_data["regr"]["panas_neg_raw_post"]["y"], COLS_KEEP, 5, classifier=False)